##### CSV and Excel files - Structured Data

In [1]:
import pandas as pd
import os


##### CSV Processing

In [4]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

##### Method:1 CSV_loader

In [7]:
csv_loader = CSVLoader(
    file_path='data/structured_files/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter':',',
        'quotechar':'"'
    }
)


csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst Document")
print(f"Content: {csv_docs[0].page_content}")
print(f"metadata: {csv_docs[0].metadata}")


Loaded 5 documents (one per row)

First Document
Content: Product: Laptop
Category: Electronics
Price: 999.99
Stock: 50
Description: High-performance laptop with 16GB RAM and 512GB SSD
metadata: {'source': 'data/structured_files/products.csv', 'row': 0}


In [8]:
print(csv_docs)

[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 4}, page_content='Product: Webcam\nCategory: Electro

###### Method:2 Custom CSV processing for better control

In [19]:
from langchain_core.documents import Document

In [20]:
def process_csv_intelligently(file_path:str):
    df = pd.read_csv(file_path)
    documents = []


    for idx,row in df.iterrows():
        content =  f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""

    
        doc = Document(
            page_content = content,
            metadata = {
                'source':file_path,
                'row_index':idx,
                'product_name':row['Product'],
                'category':row['Category'],
                'price':row['Price'],
                'data_types':'product_info'
            }
        )

        documents.append(doc)

    return documents

In [21]:
process_csv_intelligently('data/structured_files/products.csv')

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_types': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_types': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_types': 'product_info'}, page_content='Product Info

###### Excel Processing

In [24]:
def process_excel_with_pandas(file_path):
    documents = []

    excel_file = pd.ExcelFile(file_path)

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(file_path,sheet_name=sheet_name)

        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {','.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)



        doc = Document(
            page_content=sheet_content,
            metadata={
                'source':file_path,
                'sheet_name':sheet_name,
                'num_rows':len(df),
                'num_columns':len(df.columns),
                'data_types':'excel_sheet'
            }
        )
        documents.append(doc)

    return documents



In [25]:
excel_docs = process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

Processed 2 sheets


In [26]:
print(excel_docs)

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_types': 'excel_sheet'}, page_content='Sheet: Products\nColumns: Product,Category,Price,Stock,Description\nRows: 5\n\n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'), Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_types': 'excel_sheet'}, page_content='Sheet: Summary\nColumns: Category,Total_Items,T

###### Method:2 UnstructuredExcelLoader

In [27]:
from langchain_community.document_loaders import UnstructuredExcelLoader

In [29]:
try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured_files/inventory.xlsx',
        mode='elements'
    )

    unstructured_docs = excel_loader.load()


except Exception as e:
    print(e)


In [31]:
print(unstructured_docs)

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'file_directory': 'data/structured_files', 'filename': 'inventory.xlsx', 'last_modified': '2025-07-07T14:26:30', 'page_name': 'Products', 'page_number': 1, 'text_as_html': '<table><tr><td>Product</td><td>Category</td><td>Price</td><td>Stock</td><td>Description</td></tr><tr><td>Laptop</td><td>Electronics</td><td>999.99</td><td>50</td><td>High-performance laptop with 16GB RAM and 512GB SSD</td></tr><tr><td>Mouse</td><td>Accessories</td><td>29.99</td><td>200</td><td>Wireless optical mouse with ergonomic design</td></tr><tr><td>Keyboard</td><td>Accessories</td><td>79.99</td><td>150</td><td>Mechanical keyboard with RGB backlighting</td></tr><tr><td>Monitor</td><td>Electronics</td><td>299.99</td><td>75</td><td>27-inch 4K monitor with HDR support</td></tr><tr><td>Webcam</td><td>Electronics</td><td>89.99</td><td>100</td><td>1080p webcam with noise cancellation</td></tr></table>', 'languages': ['eng'], 'filetype': 'applicatio